## Setup

In [ ]:
from langgraph.prebuilt.chat_agent_executor import AgentState
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, START, END
from typing import Annotated
from dotenv import load_dotenv
import sys

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)  

## LangGraph Tools

In [ ]:
from langchain_core.messages import HumanMessage
from langchain.tools import tool

@tool
def greeting_tool() -> str:
    """Fetches a greeting message from OpenAI."""
    response = llm.invoke({"prompt": "Provide a friendly greeting message."})
    return response["choices"][0]["message"]["content"]

@tool
def farewell_tool() -> str:
    """Fetches a farewell message from OpenAI."""
    response = llm.invoke({"prompt": "Provide a polite farewell message."})
    return response["choices"][0]["message"]["content"]

@tool
def medicine_tool() -> str:
    """Fetches information about recommended medicines for common symptoms like fever or headache."""
    response = llm.invoke({"prompt": "Provide a list of recommended medicines for treating common symptoms such as fever or headache."})
    return response["choices"][0]["message"]["content"]

@tool
def medical_hospital_tool() -> str:
    """Fetches information about nearby hospitals based on general location data."""
    response = llm.invoke({"prompt": "Provide information about nearby hospitals for general medical assistance."})
    return response["choices"][0]["message"]["content"]

@tool
def medical_department_tool() -> str:
    """Fetches information about medical departments available in a hospital."""
    response = llm.invoke({"prompt": "List common medical departments in a hospital and their primary services."})
    return response["choices"][0]["message"]["content"]

## Graph \ Workflow

### Agent State

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from typing import Literal

system_prompt = (
    "You are a medical assistant providing the information related to the medical field."
    "You are also a supervisor tasked with managing a conversation between the following agents: {members}. "
    "Each agent provides specific information related to its expertise area in response to user queries. "
    "For example, MedicineAgent should give medical advice within general advice limitations, MedicalHospitalAgent should suggest hospitals based on location, and MedicalDepartmentAgent should offer relevant departmental options."
)

members = ["GreetingAgent", "FarewellAgent", "MedicineAgent", "MedicalHospitalAgent", "MedicalDepartmentAgent"]

class routeResponse(BaseModel):
    next: Literal["FINISH", "GreetingAgent", "FarewellAgent", "MedicineAgent", "MedicalHospitalAgent", "MedicalDepartmentAgent"]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next? "
            "Or should we FINISH? Select one of: ['GreetingAgent', 'FarewellAgent', 'MedicineAgent', 'MedicalHospitalAgent', 'MedicalDepartmentAgent', 'FINISH']",
        ),
    ]
).partial(members=", ".join(members))

In [ ]:
from langchain_core.messages import HumanMessage

def agent_node(state, agent, name):
    result = agent.invoke(state)
    
    if "messages" in result and result["messages"]:
        last_message = result["messages"][-1]
        message_content = last_message.content
        return {
            "messages": [HumanMessage(content=message_content, name=name)]
        }
    else:
        print(f"{name} agent did not return a message.")
        return {"messages": []}

### User Proxy Agent

In [ ]:
def UserProxyAgent(state):
    return {"messages": state["messages"]}

### Operator (Supervisor) Agent

In [8]:
def OperatorAgent(state):
    supervisor_chain = prompt | llm.with_structured_output(routeResponse)
    response = supervisor_chain.invoke(state)
    
    if hasattr(response, 'next'):
        print(f"Operator selected: {response.next}")
        return {"next": response.next, "messages": state.get("messages", [])}
    else:
        print("Error: 'next' not found in operator response.")
        return {"next": "FINISH", "messages": state.get("messages", [])}

### Specialized Agents

In [11]:
from langgraph.prebuilt import create_react_agent
import functools

GreetingsAgent = create_react_agent(llm, tools=[greeting_tool])
GreetingsNode = functools.partial(agent_node, agent=GreetingsAgent, name="GreetingAgent")

FarewellAgent = create_react_agent(llm, tools=[farewell_tool])
FarewellNode = functools.partial(agent_node, agent=FarewellAgent, name="FarewellAgent")

MedicineAgent = create_react_agent(llm, tools=[medicine_tool])
MedicineNode = functools.partial(agent_node, agent=MedicineAgent, name="MedicineAgent")

MedicalHospitalAgent = create_react_agent(llm, tools=[medical_hospital_tool])
MedicalHospitalNode = functools.partial(agent_node, agent=MedicalHospitalAgent, name="MedicalHospitalAgent")

MedicalDepartmentAgent = create_react_agent(llm, tools=[medical_department_tool])
MedicalDepartmentNode = functools.partial(agent_node, agent=MedicalDepartmentAgent, name="MedicalDepartmentAgent")

### Defining the Graph

In [ ]:
workflow = StateGraph(AgentState)
workflow.add_node("UserProxy", UserProxyAgent)
workflow.add_node("GreetingAgent", GreetingsNode)
workflow.add_node("FarewellAgent", FarewellNode)
workflow.add_node("Operator", OperatorAgent)
workflow.add_node("MedicineAgent", MedicineNode)
workflow.add_node("MedicalHospitalAgent", MedicalHospitalNode)
workflow.add_node("MedicalDepartmentAgent", MedicalDepartmentNode)

conditional_map = {
    "GreetingAgent": "GreetingAgent",
    "FarewellAgent": "FarewellAgent",
    "MedicineAgent": "MedicineAgent",
    "MedicalHospitalAgent": "MedicalHospitalAgent",
    "MedicalDepartmentAgent": "MedicalDepartmentAgent",
    "FINISH": END
}
workflow.add_conditional_edges("Operator", lambda x: x["next"], conditional_map)

workflow.add_edge(START, "UserProxy")
workflow.add_edge("UserProxy", "Operator")

graph = workflow.compile()

## Interact with User

In [ ]:
print("I am a Medical Chatbot configured by LangGraph. (type 'exit' to end with assistance)")
user_messages = []

while True:
    user_input = input("You: ")
    
    if user_input.lower() == "exit":
        print("Exiting the chatbot. Goodbye!")
        break
    
    user_messages.append(HumanMessage(content=user_input))
    
    for state in graph.stream({"messages": user_messages}):
        if "__end__" not in state:
            for agent_name, agent_response in state.items():
                if "messages" in agent_response:
                    message_content = agent_response["messages"][-1].content
                    print(f"{agent_name}: {message_content}")
                    user_messages.append(agent_response["messages"][-1])
                else:
                    print(f"No message content available in response from {agent_name}.")
            print("--------------------------------------------------------------------")